# 결손데이터 처리하기 (NaN)
## 판다스는 결손데이터(Missing Data)를 처리하는 편리한 API를 제공

In [1]:
# NULL 값을 다른 값으로 대체해야 한다. 
 # NaN 여부를 확인하는 API는 isna()이다.
 # Nan 값을 다른 값으로 대체하는 API는 fillna()이다.

In [5]:
import pandas as pd

titanic_df = pd.read_csv(r'titanic_train.csv')
print('titanic 변수 type:', type(titanic_df))
titanic_df.head(3)

#isna()로 데이터 결손여부 확인
titanic_df.isna().head(3)
# NULL 값이면 True 반환 (숫자 1)

titanic 변수 type: <class 'pandas.core.frame.DataFrame'>


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,False,False,False,False,False,False,False,False,False,False,True,False
1,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,True,False


In [7]:
# 결손데이터 개수 sum 가능
titanic_df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [9]:
# fillna()로 결손데이터 다른값으로 처리
titanic_df['Cabin'] = titanic_df['Cabin'].fillna('C000')  # Cabin 열의 결손값을 C000으로 입력처리.
titanic_df.head(3)

# inplace=True 파라미터를 fillna()에 추가해야 실제 데이터 세트 값이 변경된다.
 # -> .fillna('C000', inplace=True) 이런 식

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,C000,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,C000,S


In [10]:
titanic_df['Age'] = titanic_df['Age'].fillna(titanic_df['Age'].mean())
titanic_df['Embarked'] = titanic_df['Embarked'].fillna('S')
titanic_df.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

# Apply lambda 식으로 데이터 가공

In [12]:
def get_square(a):
    return a**2

print('3의 제곱은:', get_square(3))

3의 제곱은: 9


In [13]:
lambda_square = lambda x : x**2  # :로 계산식을 분리.   왼쪽의 x는 입력 인자, 오른쪽은 입력 인자의 계산식(즉, 반환값)
print('3의 제곱은:', lambda_square(3))

3의 제곱은: 9


In [14]:
# lambda 식 이용 시 여러 개의 값 입력 인자 사용경우엔, map()함수를 결합해서 사용.
a=[1,2,3]
squares = map(lambda x : x**2, a)
list(squares)

[1, 4, 9]

In [15]:
titanic_df['Name_len'] = titanic_df['Name'].apply(lambda x : len(x))
titanic_df[['Name', 'Name_len']].head(3)

,Name,Name_len
0,"Braund, Mr. Owen Harris",23
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",51
2,"Heikkinen, Miss. Laina",22


In [16]:
# if, else의 적용.
titanic_df['Child_Adult'] = titanic_df['Age'].apply(lambda x : 'Child' if x <= 15 else 'Adult')  # 새로운 칼럼 생성시엔 맨 앞에 새 칼럼명 작성.
titanic_df[['Age', 'Child_Adult']].head(8)

# ':' 의 오른쪽 값은 반환값이므로 반환값('child')이 먼저 나오고, if 식이 나중에 옴.
 # else의 경우는 else 식이 먼저 나오고 반환값('adult')이 나중에 옴

,Age,Child_Adult
0,22.000000,Adult
1,38.000000,Adult
2,26.000000,Adult
3,35.000000,Adult
4,35.000000,Adult
5,29.699118,Adult
6,54.000000,Adult
7,2.000000,Child


In [17]:
# apply lambda에서는 else if는 적용되지 않으므로, else 내부에 내포해 사용한다. 다중 if, else 사용시에......
titanic_df['Age_cat'] = titanic_df['Age'].apply(lambda x : 'Child' if x <= 15 else ('Adult' if x <= 60 else 'Elderly'))
#titanic_df1[['Age','Age_cat']].head(50)
titanic_df['Age_cat'].value_counts()

Adult      786
Child       83
Elderly     22
Name: Age_cat, dtype: int64

In [18]:
# 이렇게 else if가 많이 나와야하는 경우나, switch case 문 사용시에 많은 else를 내포하기엔 부담스럽다.
 # 이런경우, 별도의 함수를 만든다.

    ## 아래는 나이 세분화 분류.
def get_category(age):
    cat = ''
    if age <= 5 : cat = 'Baby'
    elif age <= 12 : cat = 'Child'
    elif age <= 18 : cat = 'Teenager'
    elif age <= 25 : cat = 'Student'
    elif age <= 35 : cat = 'Young Adult'
    elif age <= 60 : cat = 'Adult'
    else : cat='Elderly'
        
    return cat

# lambda식에 위에서 생성한 get_category() 함수를 반환값으로 설정
 # get_category(x)는 입력값으로 age칼럼을 받아 해당하는 category 반환
titanic_df['Age_spe_cat'] = titanic_df['Age'].apply(lambda x : get_category(x))
titanic_df[['Age', 'Age_spe_cat']].head(50)

,Age,Age_spe_cat
0,22.000000,Student
1,38.000000,Adult
2,26.000000,Young Adult
3,35.000000,Young Adult
4,35.000000,Young Adult
5,29.699118,Young Adult
6,54.000000,Adult
7,2.000000,Baby
8,27.000000,Young Adult
9,14.000000,Teenager
